In [62]:
#coding=utf-8

In [63]:
from PIL import Image
import numpy as np

In [64]:
def color2grey(pixel):
    return int(0.299*pixel[0] + 0.587*pixel[1] + 0.114*pixel[2])

In [65]:
def predict(testPic):
    global eigBasis
    global marvin1d
    global mean1d
    global projection
    Test1d = list(testPic.getdata())
    Test1d = [color2grey(i) for i in Test1d]
    Test1d = [(Test1d[j] - mean1d[j]) for j in range(len(marvin1d))]
    project = np.matmul(np.transpose(eigBasis), np.array(Test1d))
    costs = [cost_func(project, list(i)) for i in projection]
    #prediction = costs.index(min(costs))
    #dics = ['marvin', 'karl']
    scores = [(1- i/sum(costs)) for i in costs]
    result = {'marvin': scores[0], 'karl': scores[1]}
    return result

In [66]:
def cost_func(ls1, ls2):
    cost = 0
    for i in range(len(ls1)):
        cost += (ls1[i] - ls2[i]) ** 2
    return cost

In [69]:
#Load the original picture and convert it into array

marvin = Image.open("marvin.jpg")
karl = Image.open("karl.jpg")
width, height = marvin.size
print(width, height)
marvin.show()
karl.show()

#Turn image into array
marvin1d = list(marvin.getdata())
karl1d = list(karl.getdata())

#Convert colored to grey scale
marvin1d = [color2grey(i) for i in marvin1d]
karl1d = [color2grey(i) for i in karl1d]

#Show the grey pics
marvin2d = np.reshape(marvin1d, (-1, width))
marvin = Image.fromarray(marvin2d)
marvin.show()
marvin.save('marvin_grey', 'PNG')
karl2d = np.reshape(karl1d, (-1, width))
karl = Image.fromarray(karl2d)
karl.show()
karl.save('karl_grey', 'PNG')

50 78


In [70]:
#Merge the pics into a single array

#Pose marvin1d and karl1d together in a row-wise manner
picArray = [marvin1d, karl1d]

#Calculate the mean1d
mean1d = [(marvin1d[i] + karl1d[i]) / 2 for i in range(len(marvin1d))]

#Substract the mean1d from picArray
for i, ls in enumerate(picArray):
    picArray[i] = [(ls[j] - mean1d[j]) for j in range(len(marvin1d))]
    
#Show the substracted pics
marvinSub = Image.fromarray(np.reshape(picArray[0], (-1, width)))
karlSub = Image.fromarray(np.reshape(picArray[1], (-1, width)))
mean = Image.fromarray(np.reshape(mean1d, (-1, width)))
mean.show()
marvinSub.show()
karlSub.show()

In [71]:
#The PCA process
#Calcuate the covariance matrix of picArray

covMat = np.cov(np.transpose(picArray))  #each row is variable(pixel)

In [72]:
#The PCA process
#Calculate the eigenvalues and eigenvectors and convert into real number

eigVal, eigVec = np.linalg.eig(covMat)

In [73]:
#The PCA process
#Show the eig-faces

#Choose the top-5 eigenvalues
eigNum = 5
hiValId = []
eigVal_ls = list(eigVal)
eigVal_ls = [abs(i) for i in eigVal_ls]
for i in range(eigNum):
    maxVal = max(eigVal_ls)
    maxId = eigVal_ls.index(maxVal)
    #print(maxId, maxVal)
    hiValId.append(maxId)
    eigVal_ls[maxId] = 0

#Show the top-5 eigen-faces
for i in range(eigNum):
    eig1d = eigVec[:, hiValId[i]].real
    #Normalize the eigenvector to 255
    factor = 255 / eig1d.max()
    eig1d = np.array([i * factor for i in eig1d])
    eig2d = np.reshape(eig1d, (-1, width))
    eigFace1 = Image.fromarray(eig2d)
    eigFace1.show()

In [74]:
#The PCA process
#Create the eigen-space and use it to represent the input pics

eigBasis = np.transpose([eigVec[:, i] for i in hiValId]).real
projection = [[] for __ in range(len(picArray))]
for i, pic in enumerate(projection):
    projection[i] = np.matmul(np.transpose(eigBasis), 
                              np.transpose(picArray[i]))
print(projection)

[array([  3.94029050e+03,   2.77111667e-13,   2.77111667e-13,
         3.90798505e-13,   3.90798505e-13]), array([ -3.94029050e+03,  -2.77111667e-13,  -2.77111667e-13,
        -3.90798505e-13,  -3.90798505e-13])]


In [81]:
testPic = Image.open("balloon.jpg")
print(predict(testPic))

{'marvin': 0.27488229482959181, 'karl': 0.7251177051704083}
